<h1><center>Performance Assessment:Data Cleaning (NUM3)</center></h1>
<h3><center> by Bader Ale <center><h3>

For this Performance Assessment, I will be using the medical data contained in the D206 Definitions and Datafile directory.

# Part 1: Research Question and Variables
The research question for this analysis is:
**Is there a relation between the amount of times the primary physician visited the patient during their hospital stay and the occurence of readmission within 30 days following the patient's discharge from the facility?**

The first thing we have to do is import the original CSV file that contains our data. To do this, we must first import the necessary packages.

In [94]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell # Importing so we can run multiple lines in one cell

InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Reading in the original CSV file
df = pd.read_csv(r'C:\Users\Bader Ale\Documents\WGU_MSDA\D206_Data Cleaning\Performance Assessment\medical_raw_data.csv')

After importing our CSV file, we will see the first 5 records of our dataframe and see the overall shape/size.

In [ ]:
# Returning first 5 records of dataframe
df.head(5)

In [ ]:
# Returning number of (rows, columns)
df.shape

Our dataframe has a total of 10,000 rows and 53 columns. Next, we will return a list of all variables and their dataypes.

In [ ]:
# Return variables, datatypes and non-null status of each.
df.info()

# Part 2: Detection and Treatment of Duplicates
Our first task is to detect and treat any duplicated values in our entire dataset.

In [ ]:
# Returning a total count of duplicated values
df.duplicated().value_counts()

Here we can see there are **no** duplicated values, represented by the "False 10000" output (which is also the total rows shown in the .shape fucntion). We can now move to the next section of data cleaning, detection and treatment of missing values.

# Part 3: Detection and Treatment of Missing Values
In this section, we will see if there are any missing values for all variables in the dataset.

In [ ]:
# Returning a list of variables with total counts for missing values 
df.isnull().sum()

Here we can see that there are 7 columns with missing values; **children**, **age**, **income**, **soft_drink**, **overweight**, **anxiety** and **initial_days**
<br>
<br>1) **Children** and **Age** are considered *discrete quantitative variables* because they can only be particular numbers
<br>2) **Income** and **Initial_days** are considered *continuous quantitative variables* because your income can be whole numbers or contain decimals
<br>3) **Overweight**, **Soft_drink** and **Anxiety** are considered *nominal qualitative variables* because they are either yes or no

We will return some basic statistics on the quantitative variables to check the before and after imputation.

In [ ]:
# Checking statistical information on the columns with missing data that are quantative
df[['Children', 'Age', 'Income','Initial_days']].describe()

Using the seaborn package, we can create histograms of the quantitative variables **Children**, **Age**, **Income**, **Intial_days** to visually
analyze their distribution, but before we must import the seaborn package into our notebook.

In [ ]:
# Importing seaborn package with the inline magic function
import seaborn as sns

%matplotlib inline 

In [ ]:
#Plotting histograms
sns.displot(df, x='Children')
sns.displot(df, x='Age')
sns.displot(df, x='Income')
sns.displot(df, x='Initial_days')

From these graphs, we can see that:

1) Both **Children** and **Income** are positively skewed to the right
2) **Age** is uniformly distributed 
3) **Initial_days** has a bimodal distribution

For **Income**, **Children** and **Initial_days** variables, we will treat missing values by imputation using the median value while for **Age** we will be using the mean for imputation.

In [ ]:
# Performing imputation 
df['Children'].fillna(df['Children'].median(), inplace= True) # Using median value for Children
df['Income'].fillna(df['Income'].median(), inplace= True) # Using Median value for Income
df['Initial_days'].fillna(df['Initial_days'].median(), inplace= True) # Using median value for Initial_days
df['Age'].fillna(df['Age'].mean(), inplace= True) # Performing imputation using mean values form Age

In [ ]:
# Checking statistics again for comparison
df[['Children', 'Age', 'Income','Initial_days']].describe()

In [ ]:
# Plotting histograms again to check for skewness
#Plotting histograms
sns.displot(df, x='Children')
sns.displot(df, x='Age')
sns.displot(df, x='Income')
sns.displot(df, x='Initial_days')

Here we can see that the data behavior is relatively conserved, evident by both statistical information and histogram.
Now we will focus on the remaining categorical variables **Overweight**, **Anxiety**, and **Soft_drink**.


In [ ]:
#Detecting amount of missing values in each
print(df['Overweight'].isnull().value_counts())
print('') 
print(df['Soft_drink'].isnull().value_counts())
print('')
print(df['Anxiety'].isnull().value_counts())

Let's begin with the **Overweight** column - the **Overweight** columns is of the "0 or 1" kind. In order to treat missing values, we will calculate the percentage of each category ( 0 and 1) and impute missing values with the highest percentage category.

In [ ]:
# Determining value counts for each category in the Overweight column
df['Overweight'].value_counts(normalize=True, sort=True) # We are using the parameter normalize to return the proportions rather than frequencies

We can see a nearly 70/30 split for 1:0 ratio - this means we can impute the missing values using '1'

In [ ]:
# Treating missing values in Overweight column with '1' (highest percentage category)
df['Overweight'].fillna(1, inplace=True)

Now we will follow the same procedure for the **Anxiety** column

In [ ]:
# Determining value counts for each category in the Overweight column
df['Anxiety'].value_counts(normalize=True, sort=True) # We are using the parameter normalize to return the proportions rather than frequencies

In [ ]:
# Treating missing values in Anxiety column with '0' (highest percentage category)
df['Anxiety'].fillna(0, inplace=True)

We will now focus on the **Soft-Drink** column which is of the 'Yes/No' type - we will first re-express the variable to '0/1' using Ordinal Encoding and then using the methods used above for **Overweight** and **Anxiety** to fill is missing values.

In [ ]:
# We will run the .unique() function to determine number of unique values for specified variables
df['Soft_drink'].unique()

We see that the unique values are 'No', 'Yes' and _nan_ for missing values. We will use the percentage method mentioned above to fill these in.

In [ ]:
# Determining value counts for each category in the Overweight column
df['Soft_drink'].value_counts(normalize=True, sort=True, dropna=True) # not including the NaN values

Here we see a 75/25 split for a 1:0 ratio - we will use 'No' to fill in the missing values

In [ ]:
# Treating missing values in Soft_drink column with 'No' (highest percentage category)
df['Soft_drink'].fillna('No', inplace=True)

In [ ]:
# Rechecking for any missing values
df['Soft_drink'].unique()

We will now begin the ordinal encoding process...

In [ ]:
# Replicate the variable in preparation for replacing its categorical values with
# numeric ones. This replicated variable will store the re-expressed values once converted

df['Soft_drink_numeric'] = df['Soft_drink']

In [ ]:
# Checking duplicated column 'Soft_drink_numeric' vs original "Soft_drink"
df[['Soft_drink_numeric', 'Soft_drink']]

In [ ]:
# Set up a dictionary specifically for converting the categorical values to numeric values.
dict_soft_drink = {'Soft_drink_numeric': {'No': 0, 'Yes': 1}}

In [ ]:
# Use the dictionary to replace the variable’s values. The replacefunction will replace the values according to the rules in
# the dictionary dict_edu and store in existing data frame.
df.replace(dict_soft_drink, inplace=True)

We have detected and treated all missing values. We will now see the dataframe overall and run .info() to see if any null-values exist

In [ ]:
# Checking info on dataframe
df.info()

As one can see, there are no more variables with missing values.

# Part 4: Detection and Treatment of Outliers
In this section, we will check for any outliers in the quantitative variables and treat them accordingly.
Let's first visualize the boxplots for the variables.

In [ ]:
# Children boxplot
sns.boxplot(x='Children', data=df)

In [ ]:
# Age boxplot
sns.boxplot(x='Age', data=df)

In [ ]:
# Income boxplot
sns.boxplot(x='Income', data=df)

In [ ]:
# Initial_days boxplot
sns.boxplot(x='Initial_days', data=df)

Here we can see that **Age** and **Initial_days** have no outliers while **Children** has 4 outliers and **Income** has multiple. Since we do not know if the outliers are factual errors, we will first extract the outliers, save them as their own dataframe and then remove them from the original dataframe.

In [95]:
# Getting basic statistical info on Income and Children columns
df['Income'].describe()
df['Children'].describe()


count     10000.000000
mean      38872.450471
std       25042.796229
min         154.080000
25%       23956.162500
50%       33942.280000
75%       46466.797500
max      207249.130000
Name: Income, dtype: float64

count    10000.000000
mean         1.814000
std          1.916969
min          0.000000
25%          1.000000
50%          1.000000
75%          3.000000
max         10.000000
Name: Children, dtype: float64

We will use the z-scores to extract all records whose z-score is greater than 3. We must first import the SciPy package

In [ ]:
# Improting Scipy package
import scipy.stats as stats

In [96]:
# Creating a new column for the Income z-scores and Children z-scores
df['Income_z_Scores'] = stats.zscore(df['Income'])
df['Children_z_Scores'] = stats.zscore(df['Children'])

In [97]:
# Viewing first 10 records for both Income and Children z-score columns
df[['Income', 'Income_z_Scores']].head(10)
df[['Children', 'Children_z_Scores']].head(10)

,Income,Income_z_Scores
0,86575.93,1.904974
1,46805.99,0.316815
2,14370.14,-0.978466
3,39741.49,0.034704
4,1209.56,-1.504016
5,33942.28,-0.196880
6,10456.05,-1.134770
7,33942.28,-0.196880
8,55586.48,0.667452
9,38965.22,0.003705


,Children,Children_z_Scores
0,1.0,-0.424650
1,3.0,0.618716
2,3.0,0.618716
3,0.0,-0.946333
4,1.0,-0.424650
5,1.0,-0.424650
6,0.0,-0.946333
7,7.0,2.705447
8,1.0,-0.424650
9,2.0,0.097033


In [98]:
# Extracting records with z-scores -3 < z and z > 3 and saving as new variable 
income_outliers = df.query('Income_z_Scores < -3 | Income_z_Scores > 3')
children_outliers = df.query('Children_z_Scores < -3 | Children_z_Scores > 3')

In [99]:
# Creating a dataframe with income_outliers removed and saving as df_new
df_new = df[(df['Income_z_Scores'] > -3) & (df['Income_z_Scores'] < 3) & (df['Children_z_Scores'] > -3) & (df['Children_z_Scores'] < 3)]

In [114]:
# Checking z-scores in df_new for any z-scores missed in both Income and Children column
df_new['Children_z_Scores'].loc[lambda x : (x < -3) | (x > 3)].count()
df_new['Income_z_Scores'].loc[lambda x : (x < -3) | (x > 3)].count()

0

0